In [60]:
# import libs 
from mp_api.client import MPRester
import pandas as pd 
import numpy as np
from emmet.core.summary import HasProps
import matplotlib.pyplot as plt
import re

In [61]:
# Call the API server and download the necessary data. Refer to materialsproject.org for available fields and descriptions.
API_key = 'Your_key'
with MPRester(API_key) as mpr:
     docs = mpr.materials.summary.search(
        has_props = [HasProps.absorption], fields=["material_id", "band_gap", "formula_pretty", "chemsys", "volume","n","is_stable","efermi","is_gap_direct","is_metal","density", "absorption_coefficient"])



Retrieving SummaryDoc documents: 100%|██████████| 940/940 [00:00<?, ?it/s]


In [62]:
# store data in arrays 
mpids = [doc.material_id for doc in docs]
band_gap = [doc.band_gap for doc in docs]
formula_pretty = [doc.formula_pretty for doc in docs]
chemsys = [doc.chemsys for doc in docs]
volume = [doc.volume for doc in docs]
is_stable = [doc.is_stable for doc in docs]
efermi = [doc.efermi for doc in docs]
is_gap_direct = [doc.is_gap_direct for doc in docs]
is_metal = [doc.is_metal for doc in docs]
n = [doc.n for doc in docs]
density = [doc.density for doc in docs]

In [63]:
# insert into a dictionary 
data = {
    'mpids' : mpids,
    'band_gap': band_gap,
    'formula_pretty':formula_pretty,
    'chemsys':chemsys,
    'volume':volume,
    'is_stable':is_stable,
    'efermi':efermi,
    'is_gap_direct':is_gap_direct,
    'is_metal': is_metal,
    'density': density
}

In [64]:
# Convert to Pandas Data Frame
df =  pd.DataFrame(data)

In [65]:
df

,mpids,band_gap,formula_pretty,chemsys,volume,is_stable,efermi,is_gap_direct,is_metal,density
0,mp-546266,1.3668,DyBi2IO4,Bi-Dy-I-O,151.134541,True,4.254111,False,False,8.475086
1,mp-9583,4.3016,K2ZnF4,F-K-Zn,106.652983,True,-1.672276,False,False,3.419061
2,mp-22988,2.1535,CsGeCl3,Cl-Cs-Ge,162.925276,True,1.254099,False,False,3.178939
3,mp-861502,0.9888,AcFeO3,Ac-Fe-O,61.797311,True,6.509045,False,False,8.889999
4,mp-1025029,1.9162,PrHSe,H-Pr-Se,114.387564,True,3.203290,True,False,6.412805
...,...,...,...,...,...,...,...,...,...,...
935,mp-733,3.2501,GeO2,Ge-O,122.222447,False,-0.793378,False,False,4.264932
936,mp-1065514,0.7266,YTlTe2,Te-Tl-Y,140.002850,True,4.568730,False,False,6.505494
937,mp-7439,1.2892,K3Cu3P2,Cu-K-P,184.706554,True,2.490677,False,False,3.325279
938,mp-13276,1.3431,SrLiP,Li-P-Sr,133.068586,True,2.743818,False,False,3.133052


In [66]:
# collect absorption data form materials project API
with MPRester(API_key) as mpr:
    absorption = []
    for ids in df['mpids']:
        result = mpr.absorption.search(
            material_ids=[ids], fields=["energies", "absorption_coefficient"]
        )
        absorption.append(result)

C:\Users\AMMAR\AppData\Local\Temp\ipykernel_117832\3287923020.py:5: DeprecationWarning: Accessing absorption data through MPRester.absorption is deprecated. Please use MPRester.materials.absorption instead.
  result = mpr.absorption.search(
Retrieving AbsorptionDoc documents: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [67]:
len(absorption)

5

In [68]:
energy = []
alph = []

# Loop through the absorption data to collect energies and absorption coefficients
for i in range(len(absorption)):
    energies = absorption[i][0].energies
    abs_coeff = absorption[i][0].absorption_coefficient
    energy.append(energies)
    alph.append(abs_coeff)

# Convert the lists to 2D arrays
energy_array = np.array(energy)
alph_array = np.array(alph)


In [69]:
energy_array = np.array(energy)
alph_array = np.array(alph)

# Convert 2D arrays to pandas DataFrames
df_energy = pd.DataFrame(energy_array)
df_alpha = pd.DataFrame(alph_array)

# Add column names
df_energy.columns = [f'energy_{i}' for i in range(df_energy.shape[1])]
df_alpha.columns = [f'alpha_{i}' for i in range(df_alpha.shape[1])]


In [70]:
# Save the collected data into csv file (just to have fast access to data) use pd.read_csv('File_Name.csv') function next time to open the dataset
#df_energy.to_csv('energies_MP_940_full.csv', index=False)
#df_alpha.to_csv('alpha_MP_940_full.csv', index=False)

In [79]:
# calculate wave lenght using wl = hc/E -- we will do more processing in different notebook --
df_wl = (((4.135667696*10**-15)*(2.998*10**8))/df_energy)
df_wl.columns = [f'wl_{i}' for i in range(df_energy.shape[1])]
df_wl_nm = df_wl*10**9
df_wl_nm 

,wl_0,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,...,wl_1991,wl_1992,wl_1993,wl_1994,wl_1995,wl_1996,wl_1997,wl_1998,wl_1999,wl_2000
0,inf,44760.764450,22380.382225,14920.254817,11180.100769,8945.693905,7455.641463,6391.098842,5592.571832,4971.424119,...,22.468268,22.456995,22.445734,22.434444,22.423205,22.411977,22.400761,22.389556,22.378363,22.367140
1,inf,31074.515671,15537.257835,10358.171890,7768.628918,6214.903134,5181.250210,4440.806502,3885.531731,3453.685725,...,15.611343,15.603504,15.595673,15.587850,15.580035,15.572227,15.564427,15.556635,15.548851,15.541075
2,inf,42754.247423,21377.123711,14251.415808,10688.561856,8556.750692,7129.805493,6110.759858,5346.585491,4752.292738,...,21.482685,21.471896,21.461118,21.450351,21.439594,21.428849,21.418114,21.407426,21.396713,21.386010
3,inf,21122.200601,10561.100300,7044.733950,5282.800065,4225.879943,3521.366587,3018.191761,2641.400033,2347.799991,...,10.612227,10.606907,10.601583,10.596265,10.590952,10.585644,10.580350,10.575053,10.569761,10.564475
4,inf,33969.128089,16984.564045,11323.042696,8498.102641,6797.550303,5664.107699,4854.632636,4247.595667,3775.496880,...,17.067940,17.059368,17.050805,17.042274,17.033729,17.025191,17.016663,17.008142,16.999631,16.991128


In [72]:
element_mapping = {
    'H': 1, 'He': 2, 'Li': 3, 'Be': 4, 'B': 5, 'C': 6, 'N': 7, 'O': 8, 'F': 9, 'Ne': 10,
    'Na': 11, 'Mg': 12, 'Al': 13, 'Si': 14, 'P': 15, 'S': 16, 'Cl': 17, 'Ar': 18, 'K': 19, 'Ca': 20,
    'Sc': 21, 'Ti': 22, 'V': 23, 'Cr': 24, 'Mn': 25, 'Fe': 26, 'Co': 27, 'Ni': 28, 'Cu': 29, 'Zn': 30,
    'Ga': 31, 'Ge': 32, 'As': 33, 'Se': 34, 'Br': 35, 'Kr': 36, 'Rb': 37, 'Sr': 38, 'Y': 39, 'Zr': 40,
    'Nb': 41, 'Mo': 42, 'Tc': 43, 'Ru': 44, 'Rh': 45, 'Pd': 46, 'Ag': 47, 'Cd': 48, 'In': 49, 'Sn': 50,
    'Sb': 51, 'Te': 52, 'I': 53, 'Xe': 54, 'Cs': 55, 'Ba': 56, 'La': 57, 'Ce': 58, 'Pr': 59, 'Nd': 60,
    'Pm': 61, 'Sm': 62, 'Eu': 63, 'Gd': 64, 'Tb': 65, 'Dy': 66, 'Ho': 67, 'Er': 68, 'Tm': 69, 'Yb': 70,
    'Lu': 71, 'Hf': 72, 'Ta': 73, 'W': 74, 'Re': 75, 'Os': 76, 'Ir': 77, 'Pt': 78, 'Au': 79, 'Hg': 80,
    'Tl': 81, 'Pb': 82, 'Bi': 83, 'Po': 84, 'At': 85, 'Rn': 86, 'Fr': 87, 'Ra': 88, 'Ac': 89, 'Th': 90,
    'Pa': 91, 'U': 92, 'Np': 93, 'Pu': 94, 'Am': 95, 'Cm': 96, 'Bk': 97, 'Cf': 98, 'Es': 99, 'Fm': 100,
    'Md': 101, 'No': 102, 'Lr': 103, 'Rf': 104, 'Db': 105, 'Sg': 106, 'Bh': 107, 'Hs': 108, 'Mt': 109,
    'Ds': 110, 'Rg': 111, 'Cn': 112, 'Nh': 113, 'Fl': 114, 'Mc': 115, 'Lv': 116, 'Ts': 117, 'Og': 118
}


In [73]:
# Function to parse chemical formulas
def parse_formula(formula):
    pattern = r'([A-Z][a-z]?)(\d*)'
    matches = re.findall(pattern, formula)
    
    elements = []
    counts = []
    
    for match in matches:
        element = match[0]
        count = int(match[1]) if match[1] else 1
        elements.append(element)
        counts.append(count)
    
    return elements, counts

# Function to convert parsed results to the required format
def expand_parsed_formula(elements, counts):
    data = {}
    for i in range(len(elements)):
        # Replace element symbols with atomic numbers using element_mapping
        atomic_number = element_mapping.get(elements[i], None)
        if atomic_number is not None:
            data[f'E{i + 1}'] = atomic_number
        else:
            data[f'E{i + 1}'] = elements[i]  # fallback to original if not found in mapping
        data[f'C{i + 1}'] = counts[i]
    return data


# Apply parsing and expansion to each row
expanded_data = []
for formula in df['formula_pretty']:
    elements, counts = parse_formula(formula)
    expanded_data.append(expand_parsed_formula(elements, counts))

# Create a new DataFrame with the expanded data
expanded_df = pd.DataFrame(expanded_data)

# Combine the original DataFrame with the expanded DataFrame
result_df = pd.concat([df, expanded_df], axis=1)


In [74]:
# Save data if needed
#result_df.to_csv('MP_Data_940.csv', index= False)

In [75]:
result_df

,mpids,band_gap,formula_pretty,chemsys,volume,is_stable,efermi,is_gap_direct,is_metal,density,E1,C1,E2,C2,E3,C3,E4,C4,E5,C5
0,mp-546266,1.3668,DyBi2IO4,Bi-Dy-I-O,151.134541,True,4.254111,False,False,8.475086,66,1,83.0,2.0,53.0,1.0,8.0,4.0,NaN,NaN
1,mp-9583,4.3016,K2ZnF4,F-K-Zn,106.652983,True,-1.672276,False,False,3.419061,19,2,30.0,1.0,9.0,4.0,NaN,NaN,NaN,NaN
2,mp-22988,2.1535,CsGeCl3,Cl-Cs-Ge,162.925276,True,1.254099,False,False,3.178939,55,1,32.0,1.0,17.0,3.0,NaN,NaN,NaN,NaN
3,mp-861502,0.9888,AcFeO3,Ac-Fe-O,61.797311,True,6.509045,False,False,8.889999,89,1,26.0,1.0,8.0,3.0,NaN,NaN,NaN,NaN
4,mp-1025029,1.9162,PrHSe,H-Pr-Se,114.387564,True,3.203290,True,False,6.412805,59,1,1.0,1.0,34.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,mp-733,3.2501,GeO2,Ge-O,122.222447,False,-0.793378,False,False,4.264932,32,1,8.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
936,mp-1065514,0.7266,YTlTe2,Te-Tl-Y,140.002850,True,4.568730,False,False,6.505494,39,1,81.0,1.0,52.0,2.0,NaN,NaN,NaN,NaN
937,mp-7439,1.2892,K3Cu3P2,Cu-K-P,184.706554,True,2.490677,False,False,3.325279,19,3,29.0,3.0,15.0,2.0,NaN,NaN,NaN,NaN
938,mp-13276,1.3431,SrLiP,Li-P-Sr,133.068586,True,2.743818,False,False,3.133052,38,1,3.0,1.0,15.0,1.0,NaN,NaN,NaN,NaN


In [76]:
# replace missing values with zero (The element and count En and Cn)
result_df.fillna(0, inplace=True)

In [77]:
# Save data
#result_df.to_csv('MP_Data_940_NonNull.csv', index=False)

In [82]:
result_df

,mpids,band_gap,formula_pretty,chemsys,volume,is_stable,efermi,is_gap_direct,is_metal,density,E1,C1,E2,C2,E3,C3,E4,C4,E5,C5
0,mp-546266,1.3668,DyBi2IO4,Bi-Dy-I-O,151.134541,True,4.254111,False,False,8.475086,66,1,83.0,2.0,53.0,1.0,8.0,4.0,0.0,0.0
1,mp-9583,4.3016,K2ZnF4,F-K-Zn,106.652983,True,-1.672276,False,False,3.419061,19,2,30.0,1.0,9.0,4.0,0.0,0.0,0.0,0.0
2,mp-22988,2.1535,CsGeCl3,Cl-Cs-Ge,162.925276,True,1.254099,False,False,3.178939,55,1,32.0,1.0,17.0,3.0,0.0,0.0,0.0,0.0
3,mp-861502,0.9888,AcFeO3,Ac-Fe-O,61.797311,True,6.509045,False,False,8.889999,89,1,26.0,1.0,8.0,3.0,0.0,0.0,0.0,0.0
4,mp-1025029,1.9162,PrHSe,H-Pr-Se,114.387564,True,3.203290,True,False,6.412805,59,1,1.0,1.0,34.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,mp-733,3.2501,GeO2,Ge-O,122.222447,False,-0.793378,False,False,4.264932,32,1,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
936,mp-1065514,0.7266,YTlTe2,Te-Tl-Y,140.002850,True,4.568730,False,False,6.505494,39,1,81.0,1.0,52.0,2.0,0.0,0.0,0.0,0.0
937,mp-7439,1.2892,K3Cu3P2,Cu-K-P,184.706554,True,2.490677,False,False,3.325279,19,3,29.0,3.0,15.0,2.0,0.0,0.0,0.0,0.0
938,mp-13276,1.3431,SrLiP,Li-P-Sr,133.068586,True,2.743818,False,False,3.133052,38,1,3.0,1.0,15.0,1.0,0.0,0.0,0.0,0.0
